In [2]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:00


In [3]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.11.0


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images/255
test_images = test_images/255

In [8]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_size',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_size',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-12-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [13]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist fashion')

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 26s]
val_accuracy: 0.9088333249092102

Best val_accuracy So Far: 0.9116666913032532
Total elapsed time: 00h 03m 03s


In [15]:
model = tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 48)                743472    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 769,914
Trainable params: 769,914
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1582 - accuracy: 0.9412 - val_loss: 0.2797 - val_accuracy: 0.9075
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1202 - accuracy: 0.9548 - val_loss: 0.2978 - val_accuracy: 0.9062
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0908 - accuracy: 0.9669 - val_loss: 0.3093 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0688 - accuracy: 0.9740 - val_loss: 0.3682 - val_accuracy: 0.9110
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0538 - accuracy: 0.9801 - val_loss: 0.3765 - val_accuracy: 0.9078
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0434 - accuracy: 0.9840 - val_loss: 0.4918 - val_accuracy: 0.9083
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0342 - accuracy: 0.9877 - val_loss: 0.5160 - val_accuracy

In [24]:
model.predict(test_images)

313/313 [==============================] - 1s 2ms/step


array([[1.4258935e-20, 8.0243964e-20, 6.1601303e-19, ..., 1.2022923e-13,
        2.1769352e-26, 1.0000000e+00],
       [7.6185843e-14, 4.9331054e-18, 1.0000000e+00, ..., 1.5707815e-21,
        1.1218083e-20, 4.9159089e-21],
       [1.1465980e-29, 1.0000000e+00, 3.7080966e-33, ..., 1.5922809e-31,
        2.5493691e-27, 1.9072796e-33],
       ...,
       [6.4539457e-17, 4.8045509e-25, 3.7403363e-23, ..., 7.8108558e-24,
        1.0000000e+00, 1.0783178e-28],
       [3.2781415e-34, 1.0000000e+00, 4.8249823e-37, ..., 3.3148388e-32,
        2.5292976e-35, 2.1299305e-28],
       [3.5592208e-12, 2.9096856e-13, 2.5994285e-10, ..., 1.6461591e-04,
        4.7278075e-07, 6.8045892e-07]], dtype=float32)